In [51]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sktime.classification.interval_based import RandomIntervalSpectralEnsemble

from sklearn.metrics import classification_report, precision_recall_fscore_support 
from sklearn.model_selection import GridSearchCV, KFold

import warnings
warnings.filterwarnings('ignore')

In [52]:
X = pd.read_csv('data/lp2_X.csv')
y = pd.read_csv('data/lp2_y.csv').squeeze()

X.head()

,id,timestep,F_x,F_y,F_z,T_x,T_y,T_z
0,0,1,-2,-1,81,0,-5,0
1,0,2,-2,-1,79,0,-4,0
2,0,3,-2,-1,79,0,-4,0
3,0,4,-2,-1,80,0,-4,0
4,0,5,-3,-1,79,1,-5,1


In [53]:
y = y.apply(lambda x: 1 if x == "normal" else 0)

In [54]:
X = X.set_index(['id', 'timestep'])

In [55]:
y.value_counts()

label
0    27
1    20
Name: count, dtype: int64

In [56]:
normal_indices = y.loc[y == 1].index
collision_indices = y.loc[y == 0].index

split_idx = math.floor(len(normal_indices)/2)
train_normal_idx = pd.Index(normal_indices.tolist()[:split_idx])
test_normal_idx = pd.Index(normal_indices.tolist()[split_idx:])

split_idx = math.floor(len(collision_indices)/2)
train_col_idx = pd.Index(collision_indices.tolist()[:split_idx])
test_col_idx = pd.Index(collision_indices.tolist()[split_idx:])

train_index = pd.Index(pd.concat([pd.Series(idx) for idx in [train_normal_idx, train_col_idx]]))
test_index = pd.Index(pd.concat([pd.Series(idx) for idx in [test_normal_idx, test_col_idx]]))

In [57]:
X_train, y_train = X.loc[train_index], y.iloc[train_index]
X_test, y_test = X.loc[test_index], y.iloc[test_index]

## Random Interval Spectral Ensemble (RISE)

In [64]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train)

var_array = scaled_X_train.var(axis=0)

variance_dict = dict(zip(X_train.columns, var_array))

for column, variance in variance_dict.items():
    print(f"{column}: {variance:.4f}")

F_x: 0.0066
F_y: 0.0042
F_z: 0.0026
T_x: 0.0035
T_y: 0.0034
T_z: 0.0028


In [65]:
rise = RandomIntervalSpectralEnsemble(
    n_estimators=200,
    min_interval=10,
    acf_lag=15,
    acf_min_values=5,
    n_jobs=-1,
    random_state=42
)

uni_X_train = X_train[['F_x']]
uni_X_test = X_test[['F_x']]

rise.fit(uni_X_train, y_train)
y_pred = rise.predict(uni_X_test)

clf_report = classification_report(y_test, y_pred)

precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

print(clf_report)

              precision    recall  f1-score   support

           0       0.93      1.00      0.97        14
           1       1.00      0.90      0.95        10

    accuracy                           0.96        24
   macro avg       0.97      0.95      0.96        24
weighted avg       0.96      0.96      0.96        24

